In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#fetch themen

import requests
import json
from bs4 import BeautifulSoup
import time

URL = "https://prod.aufgabenpool.at/srdp/phpcode/search.php"


all=[]
#loop through a to z
for i in range(97, 123):
#post request with query=buchstabe as payload
    r = requests.post(url = URL, data = {'query': chr(i)})
    #extracting response text
    soup = BeautifulSoup(r.content, "html.parser")
    #get all <li> tags
    li = soup.find_all('li')

    
    #get "id" attribute of the <div> in <li> tag
    li_id = [i.find('div')['id'] for i in li]

    #get inner text of <li> tags
    li_text = [i.text for i in li]

    #remove text between and "<" until ">" 
    li_text = [i.split("<")[0] for i in li_text]

    #remove "(number)" at the end
    li_text = [i.split("(")[0] for i in li_text]

    #trim whitespaces
    li_text = [i.strip() for i in li_text]

    #combine id and text to json
    li_text = [{"id": li_id[i], "thema": li_text[i]} for i in range(len(li_id))]

    all += li_text
    print(len(all))
    time.sleep(0.2)

#remove duplicates
all = list({v['id']:v for v in all}.values())

#remove {"id": "Ooops ...", "thema": "Ihre Suche hat leider keine Treffer erzielt."}
all = [i for i in all if i['id'] != "Ooops ..."]

#save as json
with open('/content/drive/MyDrive/themen.json', 'w') as outfile:
    json.dump(all, outfile)


In [ ]:
#scrape beispiele + links

import requests
import json
from bs4 import BeautifulSoup
import time

URL = "https://aufgabenpool.at/srdp/phpcode/lp_results.php"

#read themen.json
with open('/content/drive/MyDrive/themen.json') as json_file:
    data = json.load(json_file)

res=[]
for el in data:
    payload = {'id': el['id'], 'coll': ''}

    #post request
    r = requests.post(url = URL, data = payload)

    #extracting response text
    soup = BeautifulSoup(r.content, "html.parser")

    #extract all <li> tags
    li = soup.find_all('li')

    #pdf is <a> tag
    #images can be found with src attribute from <img> tag

    all_imgs = []
    for i in li:
        a_tag = i.find('a')
        if a_tag:
          temp= {'loesung': "https://aufgabenpool.at/srdp/"+a_tag['href']}
        else:
          print(i)
          continue
        #get string between last / and .pdf
        temp['name'] = i.find('a')['href'].split("/")[-1].split(".pdf")[0]
        #find all <img> tags
        imgs = i.find_all('img')
        for i in range(len(imgs)):
            if i == 0:
                temp['aufgabe_head'] = "https://aufgabenpool.at/srdp/"+imgs[i]['src']
                temp['teil'] = imgs[i]['src'].split("teil")[1].split("/")[0].upper()
                temp['id'] = imgs[i]['src'].split("/")[1].split("/")[0].upper()
                temp["thema"]=el['thema']
            elif i == 1:
                temp['aufgabe_body'] = "https://aufgabenpool.at/srdp/"+imgs[i]['src']
                break
        res.append(temp)
    print(len(res))
    time.sleep(10)

#save as json
with open('/content/drive/MyDrive/beispiele.json', 'w') as outfile:
    json.dump(res, outfile)



In [2]:
!pip install PyPDF2 pdf2image pdfminer.six opencv-python-headless

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.8 MB/s eta 0:00:00


In [3]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (138 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 129506 fi

restart runtime

In [ ]:
#download and extract images

import cv2
import json
import sys
sys.path.append('/usr/local/lib/python3.8/dist-packages')
import requests
from PIL import Image
from io import BytesIO
import PyPDF2
import os
from pdf2image import convert_from_path
import re
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdf2image import convert_from_path
from PIL import Image
import numpy as np

def find_grey_rectangle(img_name):
    # Load the image
    img = cv2.imread(img_name)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    ret, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)


    # Find all the contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by their y-coordinate
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[1])

    # Iterate through all the contours
    for cnt in contours:
        # Find the bounding rect of the contour
        x, y, w, h = cv2.boundingRect(cnt)
        # Filter the contours based on their area and width, and if height is not less than 10% of the image height
        # (to filter out small contours that might be noise)
        if w >= 0.75*img.shape[1] and h >= 0.1*img.shape[0]:
            #check if the rectangle is not the whole image
            if x > 0 and y > 0 and x+w < img.shape[1] and y+h < img.shape[0]:
                #cut the image so only the rectangle is left
                img = img[y:y+h, x:x+w]
                break

    # Convert the cropped image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    ret, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # Find the non-zero pixels in the thresholded image
    non_zero_pixels = cv2.findNonZero(thresh)

    # Get the bounding rectangle of the non-zero pixels
    (x, y, w, h) = cv2.boundingRect(non_zero_pixels)

    # Add some padding to the bounding rectangle
    x -= 10
    y -= 10
    w += 20
    h += 20

    #error handling
    if x < 0:
        x = 0
    if y < 0:
        y = 0
    if x+w > img.shape[1]:
        w = img.shape[1] - x
    if y+h > img.shape[0]:
        h = img.shape[0] - y

    # Crop the image using the bounding rectangle coordinates
    img = img[y:y+h, x:x+w]

    # Save the cropped image
    cv2.imwrite(img_name, img)


# Loop through the pages of the PDF
def find_right_page(pdf_reader, such_string, first_match=True):
    # Initialize a variable to keep track of the number of times the string is found
    count = 0
    # Iterate through all pages
    for page_number in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_number]
        # Extract the text from the page
        text = page.extract_text()
        # Search for the string in the text
        match = re.search(such_string, text)
        if match:
            if first_match:
                return page_number+1
            else:
                count += 1
                if count == 2:
                    return page_number+1
    return -1
    
def get_string_position_and_crop(pdf_file, string, page_number):
    print("suche nach: "+string+ ", seite: " + str(page_number) + ", in: "+pdf_file)
    with open(pdf_file, 'rb') as f:
        # Create a PDF resource manager object that stores shared resources
        resource_manager = PDFResourceManager()
        # Create a PDF page aggregator object
        device = PDFPageAggregator(resource_manager, laparams=LAParams())
        # Create a PDF interpreter object
        interpreter = PDFPageInterpreter(resource_manager, device)
        # Process the specific page
        for i, page in enumerate(PDFPage.get_pages(f)):
            if i == page_number-1:
                interpreter.process_page(page)
                # Receive the LTPage object for the page
                layout = device.get_result()
                # Iterate through the layout elements
                firs=True
                pz=1
                images = convert_from_path(pdf_file, first_page=page_number, last_page=page_number)
                image = images[0]
                p2=image.height
                for element in layout:
                    if hasattr(element,'get_text'):
                        if string in element.get_text().strip():
                            print(element.get_text().strip())
                            if firs:
                              print(element.bbox[1]/layout.height)
                              firs=False
                              pz = element.bbox[1]/layout.height
                              # Convert the page to an image
                                
                                
                            else:
                                p2 = element.bbox[1]/layout.height
                                p2=image.height-p2*image.height-50

                # Crop the image based on the y value
                y1=image.height-pz*image.height-80
                if (y1) < 0:
                  y1=0
               # Handle the case where the crop parameters are outside the image
                if p2 > image.height:
                  p2=image.height
                try:
                  image = image.crop((0, y1, image.width, p2))
                  image.save(auf_dir+"/"+bsp["id"]+aufgabe+"_loesung.jpeg")
                except:
                  try:
                    image = image.crop((0, image.height, image.width, p2))
                    image.save(auf_dir+"/"+bsp["id"]+aufgabe+"_loesung.jpeg")
                  except:
                    print("Not cropped: "+ pdf_file)
    return None

#open themen.json
with open("/content/drive/MyDrive/"+'themen.json') as json_file:
    themen = json.load(json_file)

if not os.path.exists("/content/drive/MyDrive/Themen/"):
  os.makedirs("/content/drive/MyDrive/Themen/")
      #if no thema folder make one
if not os.path.exists("/content/drive/MyDrive/pdfs"):
    os.makedirs("/content/drive/MyDrive/pdfs")

for the in themen:
  #open beispiele.json
  with open("/content/drive/MyDrive/"+'beispiele.json') as json_file:
      data = json.load(json_file)
  thema=the["thema"]
  print(thema)


  #filter by thema
  data = [i for i in data if i['thema'] == thema]



  for bsp in data:
          #get "aufgabe_body"
      ab=bsp['aufgabe_body']
       #extract bsp letter + ")"
      aufgabe = ab.split("/")[-1].split(".")[0]
      aufgabe_name= bsp["name"]+"_"+bsp["id"]+aufgabe

      auf_dir="/content/drive/MyDrive/Themen/"+thema+"/"+aufgabe_name

      #if os.path.exists(auf_dir+"/"+bsp["id"]+aufgabe+"_loesung.jpeg"):
          #continue
      
      #download "loesung" as pdf
      loe=bsp['loesung']
      pdf_name= bsp["name"]+"_"+bsp["id"]+".pdf"
      #check if already downloaded
      if not os.path.exists("/content/drive/MyDrive/"+"pdfs/"+pdf_name):
          r = requests.get(loe, allow_redirects=True)
          open("/content/drive/MyDrive/"+"pdfs/"+pdf_name, 'wb').write(r.content)

     


      pdf_file = open("/content/drive/MyDrive/"+"pdfs/"+pdf_name, "rb")
      pdf_reader = PyPDF2.PdfReader(pdf_file)


      #make aufgabe_name directory in thema dir
      if not os.path.exists(auf_dir):
          os.makedirs(auf_dir)
      
      #download "aufgabe_head" as image
      ah=bsp['aufgabe_head']
      ah_name= bsp["id"]+aufgabe+"_ah"
      ah_path= auf_dir+"/"+ah_name+".png"
      #check if already downloaded
      if not os.path.exists(ah_path):
          response = requests.get(ah)
          img = Image.open(BytesIO(response.content))
          img.save(ah_path)

      #download "aufgabe_body" as image
      ab=bsp['aufgabe_body']
      ab_name= bsp["id"]+aufgabe+"_ab"
      ab_path= auf_dir+"/"+ab_name+".png"
      #check if already downloaded
      if not os.path.exists(ab_path):
          response = requests.get(ab)
          img = Image.open(BytesIO(response.content))
          img.save(ab_path)
      
      #extract solution

      such_string = aufgabe+"1)"
      reg=aufgabe+"1\)"
      such_page = None



      
      print(pdf_file)
      such_page = find_right_page(pdf_reader, reg, True)


      if such_page == -1:
          such_string = aufgabe+")"
          reg=aufgabe+"\)"
          such_page = find_right_page(pdf_reader, reg, False)

      if such_page == -1:
        such_page=len(pdf_reader.pages)-1

          

      image = get_string_position_and_crop("/content/drive/MyDrive/"+"pdfs/"+pdf_name, such_string, such_page)
      find_grey_rectangle(auf_dir+"/"+bsp["id"]+aufgabe+"_loesung.jpeg")
      
          
      pdf_file.close()
      print("\n\n")





absoluter und relativer Fehler
<_io.BufferedReader name='/content/drive/MyDrive/pdfs/B_541 Faesser (PT1_2022)_922.pdf'>
suche nach: c1), seite: 3, in: /content/drive/MyDrive/pdfs/B_541 Faesser (PT1_2022)_922.pdf
c1)  L1 = 2 ∙
0.40018722160852366
c1)   Ein Punkt für das richtige Aufstellen der Formel.
c2)   Ein Punkt für das richtige Beschreiben im gegebenen Sachzusammenhang unter Beachtung



<_io.BufferedReader name='/content/drive/MyDrive/pdfs/B_524 Attersee (PT2_2021)_892.pdf'>
suche nach: a1), seite: 4, in: /content/drive/MyDrive/pdfs/B_524 Attersee (PT2_2021)_892.pdf
a1)
0.7604513653802752



<_io.BufferedReader name='/content/drive/MyDrive/pdfs/B_508 Olympische_Sommerspiele_2008_Peking (PT1_2021)_870.pdf'>
suche nach: c1), seite: 4, in: /content/drive/MyDrive/pdfs/B_508 Olympische_Sommerspiele_2008_Peking (PT1_2021)_870.pdf
c1)   Ermittlung mittels Technologieeinsatz:
0.46647590540331874
a1)   Ein Punkt für das richtige Berechnen der Beschleunigung.
a2)   Ein Punkt für das richti

In [ ]:
!pip install genanki

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import genanki
import os

if not os.path.exists("/content/drive/MyDrive/AM_RDP_Decks"):
    os.makedirs("/content/drive/MyDrive/AM_RDP_Decks")

with open("/content/drive/MyDrive/"+'themen.json') as json_file:
    themen = json.load(json_file)
i=0
mainimgs=[]
alldecks=[]
for the in themen:
  i+=1
    #open beispiele.json
  with open('/content/drive/MyDrive/beispiele.json') as json_file:
      data = json.load(json_file)

  thema=the["thema"]

  my_deck = genanki.Deck(
      deck_id=12345, # a unique id for the deck
      name=thema+"_RDP_scraped") # the name of the deck

  maindeck = genanki.Deck(
  deck_id=i, # a unique id for the deck
  name="RDP_Scraped::"+thema.replace("/","und")) # the name of the deck
  

  img=[]
  #filter by thema
  data = [i for i in data if i['thema'] == thema]
  c=0
  
  
  for bsp in data:
      
          #get "aufgabe_body"
      ab=bsp['aufgabe_body']
       #extract bsp letter + ")"
      aufgabe = ab.split("/")[-1].split(".")[0]
      c+=1
      aufgabe_name= bsp["name"]+"_"+bsp["id"]+aufgabe
      auf_dir="/content/drive/MyDrive/Themen/"+thema+"/"+aufgabe_name
      ah=bsp['aufgabe_head']
      ah_name= bsp["id"]+aufgabe+"_ah"
      ah_path= auf_dir+"/"+ah_name+".png"
      ab=bsp['aufgabe_body']
      ab_name= bsp["id"]+aufgabe+"_ab"
      ab_path= auf_dir+"/"+ab_name+".png"
      my_card = genanki.Note(
      model=genanki.Model(
          1380120064,
    'Persie0 Style',
          fields=[
              {'name': 'Image1'},
              {'name': 'Image2'},
              
              {'name': 'AnswerImage'},
              {'name': 'Answer'},
          ],
          templates=[
              {
                  'name': 'Card 1',
                  'qfmt': '<br>{{Image1}}<br>{{Image2}}',
                  'afmt': '{{FrontSide}}<br><hr id=answer>{{AnswerImage}}<br>{{Answer}}',
              },
          ]
      ),
      fields=['<div style="text-align: center;"><img src="{}"></div>'.format(ah_name+".png"), '<div style="text-align: center;"><img src="{}"></div>'.format(ab_name+".png"), '<div style="text-align: center;"><img src="{}"></div>'.format(bsp["id"]+aufgabe+"_loesung.jpeg"), '<div style="text-align: center;"><a href="{}">Link</a></div>'.format(bsp["loesung"])])

      mainimgs.append(ah_path)
      mainimgs.append(ab_path)
      mainimgs.append(auf_dir+"/"+bsp["id"]+aufgabe+"_loesung.jpeg")
      maindeck.add_note(my_card)

      img.append(ah_path)
      img.append(ab_path)
      img.append(auf_dir+"/"+bsp["id"]+aufgabe+"_loesung.jpeg")
      my_deck.add_note(my_card)



  my_package = genanki.Package(my_deck)
  my_package.media_files = img

  alldecks.append(maindeck)
  
  

  if len(img)!=0:
    my_package.write_to_file("/content/drive/MyDrive/AM_RDP_Decks/"+thema.replace("/","und")+'.apkg')
    print(thema+" fertig")
  
package = genanki.Package(alldecks)
package.media_files = mainimgs
package.write_to_file("/content/drive/MyDrive/RDP_scraped.apkg")



absoluter und relativer Fehler fertig
Bewegungsaufgaben fertig
Binomialverteilung fertig
Änderungsmaße fertig
Differenzialrechnung fertig
Exponentialfunktion fertig
Formeln - Abhängigkeiten fertig
Grenzwert und Stetigkeit / asymptotisches Verhalten fertig
Lineare Funktion fertig
Lineare Gleichungen fertig
Lineare Gleichungssysteme fertig
Regression, Korrelation, Methode der kleinsten Quadrate fertig
Logarithmen fertig
Logarithmusfunktion fertig
Logarithmusgleichungen fertig
Normalverteilung fertig
Quadratische Funktion / Wurzelfunktion fertig
Quadratische Gleichungen fertig
Exponentialgleichungen fertig
Funktionale Zusammenhänge fertig
beschreibende Statistik fertig
Sinussatz, Cosinussatz fertig
sin, cos, tan im rechtwinkeligen Dreieck fertig
Zufallsstreubereich / Konfidenzintervall fertig
Wahrscheinlichkeit fertig
Matrizen fertig
Finanzmathematik fertig
lineare Optimierung fertig
komplexe Zahlen fertig
logarithmische Skalierung fertig
bedingte Wahrscheinlichkeit fertig
Zahlen und Maße